In [ ]:
!pip install BeautifulSoup4

In [ ]:
# importing the libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

In [ ]:
# Step 1: Sending a HTTP request to a URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [ ]:
# Step 2: Parse the html content
soup = BeautifulSoup(html_content, "lxml")

In [ ]:
# Step 3: Analyze the HTML tag, where your content lives
postalcode_table = soup.find("table", attrs={"class": "wikitable"})
postalcode_data = postalcode_table.find_all('tr')